In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,34299
2,Huelva,Confirmados PDIA 14 días,967
3,Huelva,Tasa PDIA 14 días,"188,4365804704094"
4,Huelva,Confirmados PDIA 7 días,541
5,Huelva,Total Confirmados,34486
6,Huelva,Curados,32402
7,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  34299.0
/tmp/ipykernel_66028/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9380.0
/tmp/ipykernel_66028/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_66028/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_66028/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_66028/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1050 personas en los últimos 7 días 

Un positivo PCR cada 549 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,34299.0,541.0,967.0,513170.0,105.423154,188.436580,126.0
Huelva-Costa,19860.0,374.0,664.0,289548.0,129.166839,229.322945,101.0
Huelva (capital),9380.0,137.0,262.0,143837.0,95.246703,182.150629,40.0
Isla Cristina,2329.0,97.0,138.0,21393.0,453.419343,645.070818,31.0
Condado-Campiña,11098.0,147.0,252.0,156231.0,94.091442,161.299614,21.0
Lepe,2246.0,43.0,70.0,27880.0,154.232425,251.076040,16.0
Moguer,1450.0,20.0,41.0,21867.0,91.462020,187.497142,6.0
Aljaraque,1208.0,24.0,37.0,21474.0,111.763062,172.301388,6.0
Ayamonte,1411.0,32.0,64.0,21104.0,151.630023,303.260045,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Santa Bárbara de Casa,35.0,13.0,19.0,1043.0,1246.404602,1821.668265,1.0
Niebla,281.0,32.0,32.0,4116.0,777.453839,777.453839,1.0
Isla Cristina,2329.0,97.0,138.0,21393.0,453.419343,645.070818,31.0
Alosno,207.0,11.0,24.0,3933.0,279.684719,610.221205,0.0
Zarza-Perrunal (La),31.0,2.0,5.0,1253.0,159.616919,399.042298,1.0
Puerto Moral,5.0,1.0,1.0,278.0,359.712230,359.712230,0.0
Lucena del Puerto,278.0,7.0,11.0,3261.0,214.658080,337.319841,1.0
Ayamonte,1411.0,32.0,64.0,21104.0,151.630023,303.260045,5.0
Rosal de la Frontera,36.0,2.0,5.0,1697.0,117.855038,294.637596,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,678.0,2.0,14.0,12750.0,15.686275,109.803922,1.0,0.142857
Puebla de Guzmán,145.0,1.0,4.0,3092.0,32.341527,129.366106,0.0,0.250000
Manzanilla,110.0,1.0,3.0,2118.0,47.214353,141.643059,0.0,0.333333
Sierra de Huelva-Andévalo Central,2983.0,17.0,45.0,67391.0,25.225920,66.774495,4.0,0.377778
Cartaya,1431.0,9.0,23.0,20083.0,44.814022,114.524722,1.0,0.391304
Rosal de la Frontera,36.0,2.0,5.0,1697.0,117.855038,294.637596,0.0,0.400000
Zarza-Perrunal (La),31.0,2.0,5.0,1253.0,159.616919,399.042298,1.0,0.400000
Bollullos Par del Condado,1059.0,10.0,25.0,14387.0,69.507194,173.767985,3.0,0.400000
Punta Umbría,872.0,6.0,15.0,15355.0,39.075220,97.688049,1.0,0.400000
